<a href="https://colab.research.google.com/github/Kaiziferr/deep_learning/blob/master/multilayer_perceptron/05_dropout_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install scikeras

In [27]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.constraints import MaxNorm

from scikeras.wrappers import KerasClassifier

from sklearn.model_selection import (
    cross_val_score,
    train_test_split,
    StratifiedKFold
)

from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler
)

from sklearn.pipeline import Pipeline

# **Data**
---

In [28]:
url = 'https://raw.githubusercontent.com/selva86/datasets/master/Sonar.csv'
data = pd.read_csv(url)

In [29]:
X = data.iloc[:, :60]
y = data.iloc[:, 60]

In [30]:
encoder = LabelEncoder()
y_encoder = encoder.fit_transform(y)

In [31]:
def create_model_base():
  model = Sequential()
  model.add(Dense(60, input_shape=(60, ), activation='relu'))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1, activation = 'sigmoid'))
  sgd = SGD(learning_rate=0.01, momentum=0.8)
  model.compile(loss = 'binary_crossentropy', optimizer=sgd)
  return model

In [32]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('mlp',
                   KerasClassifier(model = create_model_base, epochs = 300,
                                         batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, y_encoder, cv = kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 85.64% (7.38%)


La ejecución del ejemplo genera una precisión de clasificación estimada del 86 %.



In [33]:
def create_model_drop_input_layer():
  model = Sequential()
  model.add(Dropout(0.2, input_shape=(60,)))
  model.add(Dense(60, activation='relu', kernel_constraint=MaxNorm(3)))
  model.add(Dense(30, activation='relu', kernel_constraint=MaxNorm(3)))
  model.add(Dense(1, activation='sigmoid'))
  sgd = SGD(learning_rate=0.1, momentum=0.9)
  model.compile(loss='binary_crossentropy', optimizer=sgd)
  return model

In [34]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('mlp',
                   KerasClassifier(
                       model = create_model_drop_input_layer,
                       epochs = 300,
                       batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)

kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, y_encoder, cv = kfold)
print("Model Dropout: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Model Dropout: 85.55% (5.72%)


La ejecución del ejemplo genera una precisión de clasificación estimada del 85.55%. Mejor el rendimiento que el modelo base.

In [35]:
def create_model_drop_hidden_layer():
  model = Sequential()
  model.add(Dense(60, input_shape=(60,), activation='relu', kernel_constraint=MaxNorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(30, activation='relu', kernel_constraint=MaxNorm(3)))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  sgd = SGD(learning_rate=0.1, momentum=0.9)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

In [36]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_model_drop_hidden_layer,
                                          epochs=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, y_encoder, cv=kfold)
print("Hidden: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Hidden: 83.60% (8.51%)


No mejor, empeor un poco el rendimiento, sin embargo esto se debe a multiples situaciones. El model drop out tiene el mejor rendimiento 85.55% que el resto de modelos con una desviación menor con respecto a los otro model.